## Feb 16 2024

Topics
* Volumes vs Surfaces [pial, white, inflated] (freeview)
* fsnative vs fsaverage (T1w -> MNI)
* surface mesh vs surface data (cells below)
* plot sub 29 NC vals as overlay onto fsaverage inflated left hemisphere (freeview)
* configure -> 90%, explore
* mri_surf2surf conversion

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import nilearn
import nibabel as nib
import nilearn.surface 

from nilearn.image import load_img, mean_img
from nilearn import surface, plotting
from pathlib import Path

sub_29 = Path('/Users/alxmrphi/Documents/Data/BirdData/sub-29/')

### Volumes vs Surfaces (freeview)

### `fsnative` vs `fsaverage` 

In volume-based MRI, we have functional data and the anatomical data. The anatomical data is a high-resolution scan of a specific participant's brain. This is called the T1(w) image. The MNI template is an average of > 100 brains that we map to so that results from multiple subjects are alignable. 

In the surface world, using Freesurfer, making surfaces that are subject-specific, this is called `fsnative` space. So a specific subject's T1w VOLUME is converted into a surface that we say is in `fsnative` SPACE. 

Going from T1w to MNI (average template) in terms of surfaces is going from `fsnative` -> `fsaverage` (`fsaverage` brain is based on MNI template).

### Surfaces: mesh vs overlays

A surface is defined by two arrays:
* vertex array of coordinates
* list of triangles (pointing to triplets of vertices in the vertex array)

Can get best understanding via playing around with examples...

### Nilearn Surfaces

* `nilearn.surface.load_surf_mesh`
* `nilearn.surface.load_surf_data`

The function `load_surf_mesh` loads the surface data that has (a) list of vertex locations and (b) the triangle ("faces") triplets that say which three vertices make up a specific triangle in the mesh.

The function `load_surf_data` loads a value of interest to be plotted on each of the vertices. The triangles and structure are already set up to give the 'shape' of the brain, but the colours to be plotted on there are called 'overlays' and you lay the values over the mesh. 

The noise ceiling values are numerical values that go on top of each vertex point, so this is an 'overlay'.

In [ ]:
import nilearn
from nilearn import datasets, surface
fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage')

In [ ]:
fsaverage_infl_left = surface.load_surf_mesh(fsaverage.infl_left)
print(len(fsaverage_infl_left))
vertices, triangles = fsaverage_infl_left
print(f"{vertices.shape = }")
print(f"{triangles.shape = }")

There are 163,842 vertices in this surface, and 327,680 triangles.

Let's look at the 7th triangle.

In [ ]:
triangles[6]

This says the 7th triangle is made up of vertex 3, vertex 40975 and vertex 40977. 

Let's look at their coordinates and check they are all close together (coordinates are in millimetres)

In [ ]:
vertices[[triangles[6][0], triangles[6][1], triangles[6][2]]]

Each has similar values, as we expect. Let's do the same for a triangle in a different place.

In [ ]:
print(triangles[56564])

In [ ]:
vertices[[triangles[56564][0], triangles[56564][1], triangles[56564][2]]]

These are in a very different place (mm) from the 7th triangle.

## Overlays

If (vertices, triangles) define a surface mesh, then when we plot, we want to put a single value on each vertex in the mesh. This means an overlay should have the same number of vertices as the vertex list of the mesh and a single column value to plot. That's what the noise ceiling surface values are. 

But first, how to go from a volume of noise ceilings to a surface?

* Load the NC volumes previously calculated
* Load a surface file that `fMRIprep` calculates and has saved (we'll use `pial_lh`)
* Use the `surface.vol_to_surf` function to derive an overlay
* Use `nibabel` to save it as a GIfTI image (surface-file representation)
* Save

In [ ]:
nc_file = sub_29 / 'misc/sub-29_nc_volume.nii.gz'
#pial_lh =     Path('/Users/alxmrphi/Documents/Data/BirdData/sub-29/anat/sub-29_hemi-L_pial.surf.gii')
#inflated_lh = Path('/Users/alxmrphi/Documents/Data/BirdData/sub-29/anat/lh.inflated.gii')
white_lh =    Path('/Users/alxmrphi/Documents/Data/BirdData/sub-29/anat/sub-29_hemi-L_white.surf.gii')
#white_other =  Path('/Users/alxmrphi/Documents/Data/BirdData/birds_subjects/sub-29/surf/lh.white')

nc_img = load_img(nc_file)
nc_overlay = surface.vol_to_surf(nc_img, white_lh) 

img = nib.gifti.GiftiImage(darrays=[nib.gifti.GiftiDataArray(nc_overlay.astype(np.float32))])

# CHANGE THIS TO SAVE INTO SPECIFIC SUBJECT FOLDER and call it by the correct hemisphere, fsnative, and that its derived from white matter surface reference
nib.save(img, sub_29 / 'misc/sub29_nc_white_lh_fsnative.gii')

In [ ]:
# IGNORE
#inflated_lh_surf = nilearn.surface.load_surf_mesh(inflated_lh)
#pial_lh_surf = nilearn.surface.load_surf_mesh(pial_lh)
#white_lh_surf = nilearn.surface.load_surf_mesh(white_lh)

#print(inflated_lh_surf[0].shape)
#print(pial_lh_surf[0].shape)
#print(white_lh_surf[0].shape)

Notice this is in `fsnative` space, which is subject-specific.

Remember above we imported `fsaverage_infl_left` ? We can see how many vertices are in that mesh and compare it to subject 29 noise ceiling vertex count. 

In [ ]:
fsaverage_vertices, fsaverage_triangles = fsaverage_infl_left
print(fsaverage_vertices.shape)
print(nc_overlay.shape)


The values are different (as expected). If we want to be able to compare multiple subjects, we need to convert subject-specific overlay to one that fits on the `fsaverage` template. You do this by using a Freesurfer program called `mri_surf2surf`. This isn't available in Python and is best run from the terminal directly.

### mri_surf2surf

Need to have a folder with `fsaverage` subject and then the subject-specific Freesurfer folder for each subject (found in `/freesurfer/sourcedata/`). Then need to set environment variable $SUBJECTS_DIR to point to that location so that `mri_surf2surf` knows where to look for the data.


It should look like

    - foldername
      - fsaverage 
      - sub-01
      - sub-02
      - etc ...

And the `sourcedata` freesurfer data should be in each of those folders (not the fMRIprep outputs). Then in terminal set the environment variable SUBJECTS_DIR to `foldername` path by doing the following in the terminal:

`SUBJECTS_DIR=/path/to/foldername`

Then you're ready to run the command below (which you can do individually as not that many subjects or put it in a loop). Just be careful if doing a loop that you're not accidentally cross-loading different subjects data (by updating the source subject ID but not the sval location, for example).

    mri_surf2surf --srcsubject sub-29 \ 
                  --trgsubject fsaverage \
                  --hemi lh \
                  --sval /Users/alxmrphi/Documents/Code/src/sub29_nc_fsnative.gii \ 
                  --tval /Users/alxmrphi/Documents/Data/BirdData/sub-29/misc/sub29_nc_fsaverage.gii


`srcsubject` = source subject (here, subject 29)

`trgsubject` = target subject (here, fsaverage because we want to convert from sub-29 -> fsaverage)

`hemi` = hemisphere (everything is done per-hemisphere with surfaces because they're not modelled as being connected)

`sval` = source value (what file do we want to convert?)

`tval` = target value (what is the filename we want the output to be saved to?)

Once this runs without issue, the new file will be saved into the location specified by `tval`.

# TASK (over next week)

We want all subject data to be created on `fsnative` and `fsaverage` surfaces and saved in their data folders.

## Can ignore everything below for now, but there if you want to play around.

### Plotting with Python

* Make sure `plotly` is installed for interactive plots (otherwise defaults to static `matplotlib`)
* `!pip install plotly`

In [ ]:
inflated_lh_file = Path('/Users/alxmrphi/Documents/Data/BirdData/sub-29/surf/lh.inflated')
pial_lh = Path('/Users/alxmrphi/Documents/Data/BirdData/sub-29/anat/sub-29_hemi-L_pial.surf.gii')
white_lh = Path('/Users/alxmrphi/Documents/Data/BirdData/sub-29/anat/sub-29_hemi-L_white.surf.gii')
curve_left_file = sub_29 / 'surf' / 'lh.curv'
curv_left = surface.load_surf_data(curve_left_file)
curv_left_sign = np.sign(curv_left)
sub29_nc_fsaverage = nilearn.surface.load_surf_data(sub_29 / 'misc' / 'sub29_nc_fsaverage.gii')

In [ ]:
fig = plotting.plot_surf_stat_map(
    inflated_lh_file, nc_overlay, hemi='left',
    bg_map=curv_left_sign,
    threshold=5.5,
    engine='plotly'
)
fig.show()

In [ ]:
fig = plotting.plot_surf_stat_map(
    pial_lh, nc_overlay, hemi='left',
    bg_map=curv_left_sign,
    threshold=5.5,
    engine='plotly'
)
fig.show()

* Going to fsaverage space from fsnative
https://neurostars.org/t/transform-from-fsaverage-to-fsnative/16568

In [ ]:
nib.load()

In [ ]:
sub29_nc_fsaverage = nilearn.surface.load_surf_data(sub_29 / 'misc' / 'sub29_nc_fsaverage.gii')

In [ ]:
curv_left = surface.load_surf_data(fsaverage.curv_left)
curv_left_sign = np.sign(curv_left)

In [ ]:
from nilearn import plotting

fig = plotting.plot_surf_stat_map(
    fsaverage.infl_right, sub29_nc_fsaverage, hemi='left',
    title='Surface left hemisphere', colorbar=True,
    threshold=1., bg_map=curv_left_sign,
)
fig.show()

In [ ]:
from nilearn import plotting

fig = plotting.plot_surf_stat_map(
    fsaverage.infl_left, sub29_nc_fsaverage, hemi='left',
    title='Surface left hemisphere', colorbar=True,
    threshold=5.3, bg_map=curv_left_sign, bg_on_data=True,
    engine='plotly'
)
fig.show()

In [ ]:
glasser_lh_path = Path('/Users/alxmrphi/Downloads/lh.HCPMMP1.annot')
glasser = nilearn.surface.load_surf_data(glasser_lh_path)
glasser.shape

In [ ]:
destrieux_atlas = datasets.fetch_atlas_surf_destrieux()
parcellation = destrieux_atlas['map_right']
# these are the regions we want to outline
regions_dict = {b'G_postcentral': 'Postcentral gyrus',
                b'G_precentral': 'Precentral gyrus'}

# get indices in atlas for these labels
regions_indices = [
    np.where(np.array(destrieux_atlas['labels']) == region)[0][0]
    for region in regions_dict
]

labels = list(regions_dict.values())

In [ ]:
np.where(np.array(destrieux_atlas['labels']) == b'G_postcentral')

In [ ]:
V1_idx = 1
V2_idx = 4
V3_idx = 5
V4_idx = 6
V8_idx = 7

labels = ['V1', 'V2', 'V3', 'V4', 'V8']
regions_indices = [1,4,5,6,7]

In [ ]:
from nilearn import plotting

curve_left_file = Path('/Users/alxmrphi/Documents/Data/BirdData/birds_subjects/fsaverage/surf/lh.curv')
curv_left = surface.load_surf_data(curve_left_file)
curv_left_sign = np.sign(curv_left)


fig = plotting.plot_surf_stat_map(
    fsaverage.infl_left, sub29_nc_fsaverage, hemi='left',
    title='Surface left hemisphere', colorbar=True, view='posterior',
    threshold=5.3, bg_map=curv_left_sign, bg_on_data=True)

plotting.plot_surf_contours(fsaverage.infl_left, glasser, labels=labels,
                            levels=regions_indices, figure=fig,
                            legend=True, engine='plotly')
plotting.show()

In [ ]:
view = plotting.view_surf(fsaverage.infl_left, sub29_nc_fsaverage, threshold='98%',
                          bg_map=fsaverage.sulc_left)

view

In [ ]:
view = plotting.view_surf(fsaverage.infl_left, sub29_nc_fsaverage, threshold='98%',
                          bg_map=fsaverage.sulc_left)

plotting.plot_surf_contours(fsaverage.infl_left, glasser, labels=labels,
                            levels=regions_indices, figure=view,
                            legend=True, engine='plotly',
                            views=['lateral'])